In [1]:
!sudo yum install Xvfb -y -y
!pip install scrapy
!pip install fake_useragent
!pip install scrapy-selenium
!pip install selenium==4.9.1
!pip install boto3==1.28.5
!pip install crochet
!pip install awslambdaric
!pip install botocore==1.31.10


import os
cmd = '''
sudo yum install atk cups-libs gtk3 libXcomposite alsa-lib libXcursor libXdamage libXext libXi libXrandr libXScrnSaver libXtst pango at-spi2-atk libXt xorg-x11-server-Xvfb xorg-x11-xauth dbus-glib dbus-glib-devel -y
'''
os.system(cmd)


!curl  -Lo "/tmp/chromedriver.zip" "https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip"
!unzip /tmp/chromedriver.zip -d /opt/ -y
!sudo ln -sf /opt/chromedriver /usr/local/bin/chromedriver

!curl -Lo "/tmp/chrome-linux.zip" "https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F1135561%2Fchrome-linux.zip?alt=media"
!unzip /tmp/chrome-linux.zip -d /opt/ -y
!sudo ln -sf /opt/chrome-linux/chrome /usr/local/bin/chrome



Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
neuron                                                   | 2.9 kB     00:00     
61 packages excluded due to repository priority protections
Package xorg-x11-server-Xvfb-1.20.4-22.amzn2.x86_64 already installed and latest version
Nothing to do
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


61 packages excluded due to repository priority protections
Package atk-2.22.0-3.amzn2.0.2.x86_64 already installed and latest version
Package 1:cups-libs-1.6.3-51.amzn2.0.2.x86_64 already installed and latest version
Package gtk3-3.22.30-3.amzn2.x86_64 already installed and latest version
Package libXcomposite-0.4.4-4.1.amzn2.0.2.x86_64 already installed and latest version
Package alsa-lib-1.1.4.1-2.amzn2.x86_64 already installed and latest version
Package libXcursor-1.1.15-1.amzn2.x86_64 already installed and latest version
Package libXdamage-1.1.4-4.1.amzn2.0.2.x86_64 already installed and latest version
Package libXext-1.3.3-3.amzn2.0.2.x86_64 already installed and latest version
Package libXi-1.7.9-1.amzn2.0.2.x86_64 already installed and latest version
Package libXrandr-1.5.1-2.amzn2.0.3.x86_64 already installed and latest version
Package libXScrnSaver-1.2.2-6.1.amzn2.0.2.x86_64 already installed and latest version
Package libXtst-1.2.3-1.amzn2.0.2.x86_64 already installed and la

In [2]:
!ls -al /usr/local/bin/chrome
!ls -al /usr/local/bin/chromedriver
!ls -al /opt/chrome-linux/chrome

lrwxrwxrwx 1 root root 24 Aug 25 02:38 /usr/local/bin/chrome -> /opt/chrome-linux/chrome
lrwxrwxrwx 1 root root 17 Aug 25 02:38 /usr/local/bin/chromedriver -> /opt/chromedriver
-rwxr-xr-x 1 ec2-user ec2-user 358375680 Apr 25 22:16 /opt/chrome-linux/chrome


In [3]:
import datetime
import json
import os
import botocore
from fake_useragent import UserAgent
import boto3
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from crochet import setup
import threading
# from momoshop_link import MomoshopSpider

# Initialize Crochet
setup()

ua = UserAgent()
CHROME_BROWSER_PATH = '/usr/local/bin/chrome'
region = 'ap-northeast-1'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
queue_tier4_links = 'https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'

### spiders.momoshop

In [4]:
import hashlib
import json
import os
import threading
from urllib.parse import urlparse

from scrapy import Selector
from scrapy_selenium import SeleniumRequest
import botocore
import scrapy
from fake_useragent import UserAgent
import boto3
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver



ua = UserAgent()
CHROME_BROWSER_PATH = '/usr/local/bin/chrome'
sqs = boto3.client('sqs', region_name=region, config=botocore.client.Config(max_pool_connections=500))
s3 = boto3.client('s3', region_name=region, config=botocore.client.Config(max_pool_connections=500))
region = 'ap-northeast-1'
export_to_s3 = 'Off'
queue_tier4_content_links =  '	https://sqs.ap-northeast-1.amazonaws.com/407620147666/aws_crawler_tier4_links.fifo'





def hash_function(original_string):
    # Create a hash object using SHA-256 algorithm
    hasher = hashlib.sha256()

    # Convert the input string to bytes (required by hashlib)
    input_bytes = original_string.encode('utf-8')

    # Update the hash object with the input bytes
    hasher.update(input_bytes)

    # Get the hexadecimal representation of the hash value
    return hasher.hexdigest()


class MomoshopSpider(scrapy.Spider):
    name = "momoshop"
    allowed_domains = ["www.momoshop.com.tw"]
    user_agent = ua.random
    runner = None
    category_links = []
    tier = None

    def __init__(self, runner=None, **kwargs):
        self.runner = runner
        super(MomoshopSpider, self).__init__(**kwargs)

    def start_requests(self):
        # Start the initial request to fetch category links
        print("Starting start_requests")
        
        if self.runner.target_url == "":
            tier4_content_link = json.loads(self.runner.tier4_content_object['Body'])['tier4_content_link']
        else:
            tier4_content_link = self.runner.target_url

        # tier4_content_link = 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593'
        # self.tier = {'tier1': '個人清潔', 'tier2': '洗髮精/潤護髮', 'tier3': '精選品牌', 'tier4': 'FineToday日系髮品'}


        headers = {"Host": urlparse(tier4_content_link).netloc,
                   'Accept-Encoding': 'gzip, deflate, br',
                   'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5',
                   'Sec-Fetch-Dest': 'document',
                   'Sec-Fetch-Mode': 'navigate',
                   'Sec-Fetch-Site': 'none',
                   'Upgrade-Insecure-Requests': '1',
                   "Referer": "https://www.momoshop.com.tw/main/Main.jsp",
                   "User-Agent": ua.random}
        print(f'tier4_content_link: {tier4_content_link}')
        # https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10670127&str_category_code=1501202193&ctype=B&Area=DgrpCategory&sourcePageType=4
        yield SeleniumRequest(url=tier4_content_link,
                              headers=headers,
                              callback=self.parse,
                              wait_time=15,
                              errback=self.error_handle,
                              wait_until=ec.presence_of_element_located((By.XPATH, "//*"))
                              )
        


    @staticmethod
    def set_all_tier(response):
        # The original class array => ['Home', '家電', '飲水設備', '\n            ', '本月主打', '元山★開館慶下殺']
        tier_array = "|".join(response.css("#bt_2_layout_NAV ul li ::text").getall()).replace(" ", "").replace("\n", "").replace("||", "|").split("|")
        tier_array.pop(0)
        print(tier_array)
        return tier_array

#     @staticmethod
#     def construct_s3_tier_folder(content_tier: dict):
#         # Construct the base folder path
#         base_path = "content"

#         # Create the base folder if it doesn't exist
#         if base_path:
#             s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{base_path}/")

#         # Construct the folder structure based on the tiers
#         for key, value in content_tier.items():

#             folder_name = value.strip('/')
#             folder_path = f"{base_path}/{folder_name}"

#             # Create the folder if it doesn't exist
#             s3.put_object(Bucket=s3_crawler_content_folder, Key=f"{folder_path}/")

#             # Set the current folder path as the base path for the next iteration
#             base_path = folder_path

    def error_handle(self, error):
        print(error)
        self.runner.timeout = True

    def parse(self, response, **kwargs):
        try:
            # Parse the main category page and extract links to individual product pages
            print(f"Starting to parse")
            selector = Selector(response)
            
            # print(response.request.meta)
            # driver = response.request.meta['driver']
            
            options = webdriver.ChromeOptions()
            options.binary_location = "/opt/chrome-linux/chrome"
            options.add_argument("--headless")
            options.add_argument("--no-sandbox")
            options.add_argument("--single-process")
            options.add_argument("--disable-dev-shm-usage")
            options.add_argument("blink-settings=imagesEnabled=false")
            options.add_argument("--remote-debugging-port=9222")
            options.add_argument("--no-zygote")
            options.add_argument("--disable-gpu")
            options.add_argument("--disable-dev-tools")
    
            driver = webdriver.Chrome(executable_path = '/usr/local/bin/chromedriver', chrome_options=options)
            driver.get(response.url)

            content_info_tier = self.set_all_tier(response)
            # self.construct_s3_tier_folder(content_info_tier)

            # 品牌名稱
            item_brand = selector.css("#webBrand ::text").get(None)

            # 市售價
            market_price = None

            # 促銷價
            promotional_price = None

            # 品牌名稱
            discounted_price = None

            # 折扣後價格
            price_list = selector.css(".prdPrice li")

            for price in price_list:
                if price.css("li ::text").get() == '市售價':
                    market_price = int(price.css("li .seoPrice ::text").get().replace(',', ''))
                if price.css("li ::text").get() == '促銷價':
                    promotional_price = int(price.css("li .seoPrice ::text").get().replace(',', ''))
                if price.css("li ::text").get() == '折扣後價格':
                    discounted_price = int(price.css("li .seoPrice ::text").get().replace(',', ''))
            
            # <span id="osmGoodsName"> <a class="productName" href="/search/searchShop.jsp?keyword=%E6%9E%97%E5%85%A7&amp;brand=%E6%9E%97%E5%85%A7&amp;brandNo=20160808160045893">林內</a> 台爐式內焰二口爐輕量爐架(RTS-N201S原廠安裝)</span>
            # 商品名稱
            item_name = "".join(selector.css("#osmGoodsName ::text").extract()).strip()
            # 活動
            activity = "".join(selector.css(".ineventArea li ::text").extract()).replace(' ', '').replace('\n', '')
            print(activity)
            # 商品規格
            item_spec = {}
            for item_spec_attribute in selector.css("#attributesTable tr"):
                item_spec[item_spec_attribute.css("th ::text").get()] = str(item_spec_attribute.css("td ::text").get()).replace(' ', '')
                
            # 相關類別
            related_category_list = []
            for related_category_elements in selector.css(".related_category dl"):
                if related_category_elements.css("dl::attr(class)").get() != "brandTxt":
                    related_category_list.append("|".join(related_category_elements.css("dd ::text").getall()).replace("\\\\", "\\"))
            # 商品評價
            goods_commend = {"count": int(selector.css('.goodsCommendLi span::attr(goodscount)').get("5").replace(",", ""))}

            # 評論內容
            review_card_list = []

            # If there's any goods commend then fetch all of them
            if goods_commend["count"] > 0:
                # Send the javascript to click
                js_code = f"document.querySelector('.goodsCommendLi').click();"  # Click the page number
                driver.execute_script(js_code)

                # Wait for the new content to load after the click
                # Adjust the timeout as needed
                # wait = WebDriverWait(driver, 15)
                # # # Wait for goods comment is ready for click
                # wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, '.reviewCard')))

                # Get the updated page source and extract product links
                updated_content = driver.page_source
                selector = Selector(scrapy.http.HtmlResponse(url=driver.current_url,
                                                             body=updated_content,
                                                             encoding='utf-8'))

                goods_commend["indicator_average_value"] = float(selector.css('.indicatorAvgVal::text').get())

                for indicator in selector.css(".indicator"):
                    if indicator.css(".indicatorTitle::text").get() == "商品品質":
                        goods_commend["indicator_quality"] = float(selector.css('.indicatorNumber::text').get(None))
                    if indicator.css(".indicatorTitle::text").get() == "商品符合":
                        goods_commend["indicator_accurate"] = float(selector.css('.indicatorNumber::text').get(None))
                    if indicator.css(".indicatorTitle::text").get() == "出貨速度":
                        goods_commend["indicator_shipping"] = float(selector.css('.indicatorNumber::text').get(None))

                # Fetch all commends info
                # 商品評論統計
                goods_commend_info = []
                for goods_commend_info_attribute in selector.css(".SelectorOption"):
                    goods_commend_info.append("".join(goods_commend_info_attribute.css("::text").getall()))
                goods_commend["goods_commend_info"] = goods_commend_info

                # Fetch the all comments in the first page
                self.fetch_goods_commend(selector, review_card_list)
                # 上一頁/下一頁
                page_control = selector.css('.pageArea dd')
                if len(page_control) > 0:  # Has next page
                    has_next_page = True
                    while has_next_page:
                        first_page_control = page_control[0]
                        second_page_control = page_control[1] if len(page_control) > 1 else None
                        if first_page_control.css('a::text').get() == "下一頁":
                            # Get the next page index
                            page_index = first_page_control.css("dd::attr(pageidx)").get()
                            print(f'page_index: {page_index}')
                            page_control = self.click_to_next_page(page_index, driver, review_card_list)
                        elif second_page_control is None:
                            has_next_page = False
                        else:
                            # Get the next page index
                            page_index = second_page_control.css("dd::attr(pageidx)").get()
                            print(f'page_index:{page_index}')
                            page_control = self.click_to_next_page(page_index, driver, review_card_list)

            goods_commend["review_card_list"] = review_card_list

            content_info = {'item_brand': item_brand,
                            'item_name': item_name,
                            'market_price': market_price,
                            'promotional_price': promotional_price,
                            'discounted_price': discounted_price,
                            'activity': activity,
                            'data_date': datetime.datetime.now().strftime("%Y%m%d"),
                            'item_url': response.url,
                            'item_spec': item_spec,
                            'related_category_list': related_category_list,
                            'goods_commend_indicator_average_value': goods_commend["indicator_average_value"],
                            'goods_commend_goods_commend_info': goods_commend["goods_commend_info"],
                            'goods_commend_indicator_quality': goods_commend["indicator_quality"],
                            'goods_commend_indicator_accurate': goods_commend["indicator_accurate"],
                            'goods_commend_indicator_shipping': goods_commend["indicator_shipping"],
                            'goods_commend_review_card_list': goods_commend["review_card_list"]
                            }
            print(content_info)
            # Configure tiers
            for i in range(0, len(content_info_tier)):
                content_info[f'tier{i + 1}'] = content_info_tier[i]

            # Send the result json to s3
            # hash_id = hash_function(response.url)
            # self.send_to_s3(content_info, hash_id)

            # output the result in the local
            with open('output.json', 'w') as json_file:
                json_file.write(json.dumps(content_info, ensure_ascii=False))

        except Exception as e:
            print(str(e))
    def click_to_next_page(self, page_index: str, driver, review_card_list):
        # Send the javascript to click
        js_code = f"[...document.querySelectorAll('.pageArea a')].find(a => a.textContent.trim() === '{page_index}').click();"   # Click the page number
        driver.execute_script(js_code)

        # Get the updated page source and extract product links
        updated_content = driver.page_source
        updated_selector = Selector(scrapy.http.HtmlResponse(url=driver.current_url,
                                                             body=updated_content,
                                                             encoding='utf-8'))

        self.fetch_goods_commend(updated_selector, review_card_list)
        return updated_selector.css('.pageArea dd')

    @staticmethod
    def fetch_goods_commend(selector: Selector, review_card_list):
        for review_card in selector.css('.reviewCard'):
            review_card_score = review_card.css(".RatingStarGroup::attr(score)").get()
            review_card_spec = "".join(review_card.css(".SpecValue ::text").getall())
            review_card_date = review_card.css(".Info ::text").get()
            review_card_comment = review_card.css(".CommentContainer ::text").get()
            review_card_list.append({
                'review_card_score': review_card_score,
                'review_card_spec': review_card_spec,
                'review_card_date': review_card_date,
                'review_card_comment': review_card_comment
            })


#     @staticmethod
#     def send_to_s3(message, hash_id):
#         # Send the message to S3
#         try:
#             json_file_name = f'{hash_id}.json'

#             # Construct the S3 key including the folder name
#             s3_key = f'content/{message["tier1"]}/{json_file_name}'

#             response = s3.put_object(Bucket=s3_crawler_content_folder, Key=s3_key,
#                                      Body=json.dumps(message, ensure_ascii=False))
#             # check if it's successful
#             if response["ResponseMetadata"]["HTTPStatusCode"] != 200:
#                 print('Fail')
#         except Exception as e:
#             print(str(e))


    def spider_closed(self):
        # Handle the spider closing event
        self.runner.timeout = False

In [5]:
class LambdaRunner:
    target_url = ""
    receipt_handle = ""
    tier4_content_object = None
    timeout = False

    def __init__(self, url):
        self.finished = threading.Event()
        self.results = []
        if url != "":
            self.target_url = url

    def run_spider(self):
        # Create a CrawlerRunner with project settings
        settings = get_project_settings()
        runner = CrawlerRunner(settings)
        if self.target_url == "":
            self.get_tier4_content_url_from_sqs()

        # Callback function to handle the spider results
        def handle_results(result):
            self.results.append(result)

            # Check if the spider has finished running
            if len(self.results) == 1:
                self.finished.set()

        # Start the first spider run
        deferred = runner.crawl(MomoshopSpider, self)
        deferred.addCallback(handle_results)

        # Start the reactor
        runner.join()

    def wait_for_completion(self):
        self.finished.wait()

    def get_results(self):
        return self.results

    def get_tier4_content_url_from_sqs(self):
        response = sqs.receive_message(
            QueueUrl=queue_tier4_content_links,
            MaxNumberOfMessages=1,  # Retrieve 10 messages
            WaitTimeSeconds=0  # Maximum time to wait for messages (long polling)
        )
        messages = response.get('Messages', [])
        if messages is not None:
            self.tier4_content_object = messages[0]
        else:
            msg = "All consumed."
            print(msg)
            return msg

        # Delete messages from SQS
        for message in messages:
            sqs.delete_message(
                QueueUrl=queue_tier4_content_links,
                ReceiptHandle=message['ReceiptHandle']
            )


def handler(event, context):
    try:
        # Check if the function was triggered by an HTTP request or Lambda event
        print(f"Starting to crawl:{datetime.datetime.now()}")
        times = 0
        if "statusCode" not in event:
            # If the function was not triggered by retry
            runner = LambdaRunner("")
            runner.run_spider()
            runner.wait_for_completion()
            print(f"End date and time:{datetime.datetime.now()}")
        else:
            times = int(event["times"])
            if times < 4:
                runner = LambdaRunner(event["category_link"])
                runner.input_url = event["category_link"]
                runner.run_spider()
                runner.wait_for_completion()
                print(f"End date and time:{datetime.datetime.now()}")
            else:
                print(f'Retry too many times, 429:{event["category_link"]}')
                # If the retry count is 4 or more, return an HTTP 429 response indicating Too Many Requests
                return {
                    'statusCode': 429,
                    'body': "",
                    'times': times,
                    "category_link": event["category_link"]
                }

        times = times + 1
        if not runner.timeout:
            print("success")
            # If the LambdaRunner completed successfully, return an HTTP 200 response with the completion details
            return {
                'statusCode': 200,
                'body': 'Completed!',
                'times': times
            }
        else:
            print('timeout')
            # If the LambdaRunner timed out, return an HTTP 408 response with the category objects
            return {
                'statusCode': 408,
                'times': times,
                "category_link": json.loads(runner.tier4_content_object['Body'])['category_link']
            }
    except Exception as e:
        print(f'fail:{e}')
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

#
if __name__ == '__main__':
    handler({'statusCode': 408,'times': 1, 'category_link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593'}, "")



Starting to crawl:2023-08-25 02:38:47.603890
Starting start_requests
tier4_content_link: https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593
Starting to parse
['個人清潔', '洗髮精/潤護髮', '精選品牌', 'FineToday日系髮品']
08/23~08/31 結帳9折↘ 滿1件享9折(說明)08/23~08/31 滿888折100↘滿888再折100(說明)
page_index: 2
page_index:3
page_index:4
{'item_brand': 'TSUBAKI 思波綺', 'item_name': 'TSUBAKI 思波綺 升級版 瞬亮洗髮/潤髮 490ml(任選1入)', 'market_price': 290, 'promotional_price': 274, 'discounted_price': 246, 'activity': '08/23~08/31\xa0結帳9折↘\xa0滿1件享9折(說明)08/23~08/31\xa0滿888折100↘滿888再折100(說明)', 'data_date': '20230825', 'item_url': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593', 'item_spec': {'品牌名稱': 'TSUBAKI思波綺', '香味': '果香', '品牌定位': '沙龍', '包裝組合': '單入組', '功效': '柔亮滑順', '容量': '401~600ml', '適用於': '所有髮質', '

In [6]:
handler({'statusCode': 408,'times': 1, 'category_link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593'},
         "")

Starting to crawl:2023-08-25 02:39:00.331086
Starting start_requests
tier4_content_link: https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593
Starting to parse
['個人清潔', '洗髮精/潤護髮', '精選品牌', 'FineToday日系髮品']
08/23~08/31 結帳9折↘ 滿1件享9折(說明)08/23~08/31 滿888折100↘滿888再折100(說明)
page_index: 2
page_index:3
page_index:4
{'item_brand': 'TSUBAKI 思波綺', 'item_name': 'TSUBAKI 思波綺 升級版 瞬亮洗髮/潤髮 490ml(任選1入)', 'market_price': 290, 'promotional_price': 274, 'discounted_price': 246, 'activity': '08/23~08/31\xa0結帳9折↘\xa0滿1件享9折(說明)08/23~08/31\xa0滿888折100↘滿888再折100(說明)', 'data_date': '20230825', 'item_url': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10795343&cid=recitri&oid=BfL&mdiv=category_momoshop-cap_p2-&ctype=B&recomd_id=rgc-hqlk_normal_1692180696_195593', 'item_spec': {'品牌名稱': 'TSUBAKI思波綺', '香味': '果香', '品牌定位': '沙龍', '包裝組合': '單入組', '功效': '柔亮滑順', '容量': '401~600ml', '適用於': '所有髮質', '

{'statusCode': 200, 'body': 'Completed!', 'times': 2}